# Exemplo de Simulação de Extinção em Cascata

Este notebook demonstra como usar a classe `ExtincaoEmCascata` para simular extinções de espécies em redes ecológicas.

In [1]:
import igraph as ig
import matplotlib.pyplot as plt
from cascading_extinction import ExtincaoEmCascata

## Carregar o Grafo da Rede Trófica

In [9]:
# Carrega o grafo da rede trófica
g = ig.Graph.Read_GraphML('data/grafos/Arctic Shelf Area, Arctic.graphml')
print(f"Grafo carregado: {len(g.vs)} espécies, {len(g.es)} interações")

Grafo carregado: 227 espécies, 1065 interações


## Criar Instância da Simulação

In [10]:
# Inicializa a simulação
sim = ExtincaoEmCascata(g)

print(f"Espécies basais (grau de entrada=0): {len(sim.especies_basais)}")
print(f"\nPrimeiras 10 espécies basais:")
for idx in list(sim.especies_basais)[:10]:
    v = sim.grafo_original.vs[idx]
    print(f"  - {v['name']}")

Espécies basais (grau de entrada=0): 138

Primeiras 10 espécies basais:
  - Nephtys ciliata
  - Nereimyra punctata
  - Priapulus caudatus
  - Phyllodoce
  - Leitoscoloplos mammosus
  - Heteromastus filiformis
  - Levinsenia gracilis
  - Lysippe labiata
  - Pholoe assimilis
  - Ampharete finmarchica


## Exemplo 1: Remover uma Única Espécie

In [11]:
# Remove uma espécie com alto betweenness (da sua análise)
result = sim.remove_especie('Xanthocalanus')

print(f"Extinção primária: {result['primary_extinction']}")
print(f"Extinções secundárias: {result['total_extinctions'] - 1}")
print(f"Total de extinções: {result['total_extinctions']}")
print(f"Espécies restantes: {result['remaining_species']}")

if result['secondary_extinctions']:
    print(f"\nEspécies que foram extintas devido à cascata:")
    for species in result['secondary_extinctions']:
        print(f"  - {species}")

Extinção primária: Xanthocalanus
Extinções secundárias: 154
Total de extinções: 155
Espécies restantes: 72

Espécies que foram extintas devido à cascata:
  - Laminaria digitata
  - Cossura longocirrata
  - Scoletoma fragilis
  - Mya truncata
  - Semibalanus balanoides
  - Boremysis arctica
  - Caprella septentrionalis
  - Pleusymtes glabroides
  - Rhachotropis inflata
  - Melita formosa
  - Serripes groenlandicus
  - Euspira pallida
  - Laminaria saccharina
  - Thelepus cincinnatus
  - Boreacola maltzani
  - Prionospio cirrifera
  - Balanus balanus
  - Boreomysis arctica
  - Stilomysis grandis
  - Eryhtrops erythrophthalma
  - Spirontocaris turgida
  - Diastylis goodsiri
  - Diastylis lucifera
  - Corophium crassicorne
  - Styela rustica
  - Hiatella arctica
  - Saccorhiza dermatodea
  - Terebellides stroemi
  - Halicryptus spinulosus
  - Axinopsida orbiculata
  - Themisto libellula
  - Anonyx sarsi
  - Mysis oculata
  - Melita palmata
  - Themisto abyssorum
  - Gammarellus homari
  - 

## Reiniciar e Tentar Outra Espécie

In [8]:
# Reinicia a simulação
sim.reset()
print(f"Simulação reiniciada. Contagem de espécies: {len(sim.grafo.vs)}")

Simulação reiniciada. Contagem de espécies: 227


## Exemplo 2: Simular uma Sequência de Remoções

In [ ]:
# Remove as 5 principais espécies por centralidade de betweenness
top_species = [
    'Calanus finnmarchicus',
    'Boreogadus saida',
    'Lebbeus polaris',
    'Lumpenus lampretaeformis',
    'Liparis liparis'
]

results = sim.simular_sequencia(top_species)

print("Resumo da cascata de extinção:")
print("="*60)
for i, result in enumerate(results, 1):
    print(f"\nPasso {i}: Removido {result['primary_extinction']}")
    print(f"  Extinções secundárias: {len(result['secondary_extinctions'])}")
    print(f"  Total de extinções neste passo: {result['total_extinctions']}")
    print(f"  Espécies restantes: {result['remaining_species']}")

## Obter Resumo da Simulação

In [ ]:
summary = sim.get_resumo()

print("Resumo da Simulação:")
print("="*60)
print(f"Total de passos: {summary['total_steps']}")
print(f"Extinções primárias: {summary['total_primary_extinctions']}")
print(f"Extinções secundárias: {summary['total_secondary_extinctions']}")
print(f"Total de extinções: {summary['total_extinctions']}")
print(f"Espécies originais: {summary['original_species_count']}")
print(f"Espécies restantes: {summary['remaining_species_count']}")
print(f"Robustez da rede: {summary['robustness']:.2%}")

## Exemplo 3: Ranking de Vulnerabilidade

Encontre quais espécies, se removidas, causariam as maiores extinções em cascata.

In [ ]:
# Reinicia para análise nova
sim.reset()

# Isso pode demorar um pouco para grafos grandes
print("Calculando ranking de vulnerabilidade...")
vulnerability = sim.get_vulnerabilidade_ranking()

print("\n20 espécies mais críticas (por tamanho de cascata):")
print("="*60)
for i, (species, cascade_size) in enumerate(vulnerability[:20], 1):
    print(f"{i:2d}. {species:50s} -> {cascade_size:3d} extinções")

## Visualizar Cascata de Extinção

In [ ]:
# Reinicia e executa uma nova simulação para plotagem
sim.reset()

# Remove as 10 principais espécies por betweenness
top_10_species = [
    'Calanus finnmarchicus',
    'Boreogadus saida',
    'Lebbeus polaris',
    'Lumpenus lampretaeformis',
    'Liparis liparis',
    'Gadus morhua',
    'Hyas araneus',
    'Thysanoessa raschii',
    'Crossaster papposus',
    'Anisarchus medius'
]

results = sim.simular_sequencia(top_10_species)

# Extrai dados para plotagem
steps = [r['step'] for r in results]
cumulative_extinctions = []
total = 0
for r in results:
    total += r['total_extinctions']
    cumulative_extinctions.append(total)

# Plota
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.bar(steps, [r['total_extinctions'] for r in results], color='steelblue')
plt.xlabel('Passo de Remoção')
plt.ylabel('Total de Extinções por Passo')
plt.title('Extinções por Passo de Remoção')
plt.grid(axis='y', alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(steps, cumulative_extinctions, marker='o', linewidth=2, markersize=8, color='darkred')
plt.xlabel('Passo de Remoção')
plt.ylabel('Extinções Cumulativas')
plt.title('Cascata de Extinção Cumulativa')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nRobustez final: {sim.get_robustez():.2%}")

## Exemplo 4: Verificar Proteção de Espécies Basais

Demonstra que espécies basais (grau de entrada=0) não são extintas por cascata.

In [ ]:
# Reinicia a simulação
sim.reset()

# Obtém uma espécie basal
basal_idx = list(sim.especies_basais)[0]
basal_species = sim.grafo_original.vs[basal_idx]

print(f"Exemplo de espécie basal: {basal_species['name']}")
print(f"Grau de entrada: {basal_species.indegree()}")
print(f"Grau de saída: {basal_species.outdegree()}")

# Encontra uma espécie que come esta espécie basal
predators = sim.grafo_original.neighbors(basal_idx, mode='out')

if predators:
    predator = sim.grafo_original.vs[predators[0]]
    print(f"\nRemovendo predador: {predator['name']}")
    
    result = sim.remove_especie(predator.index)
    
    # Verifica se a espécie basal ainda existe
    basal_still_exists = any(v['name'] == basal_species['name'] for v in sim.grafo.vs)
    
    print(f"\nEspécie basal '{basal_species['name']}' ainda existe: {basal_still_exists}")
    print(f"Isto está correto - espécies basais NÃO devem ser extintas por cascata!")
else:
    print("\nEsta espécie basal não tem predadores (produtor de nível superior).")